### Importing all relevant libraries

In [1]:
%matplotlib inline
from Functions import A1, A2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the functions

In [3]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])

run = 50

functions = [A1, A2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

df3 = pd.read_csv('2019.03.05_scrambled.csv').T.iloc[1:].T

df4 = pd.DataFrame(sobol_seq.i4_sobol_generate(2*k, run*3*2**(13-1)))

df = pd.read_csv('2019.04.05.csv',index_col=1)

df5 = df4.reindex(df.index).reset_index(drop=True)

### Run the algorithm

In [3]:
n = 2

for p in range(5,14):
    run_samples = []

    MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    for r in range (run):
        run_samples.append(df5.iloc[r*3*2**(p-1):(r+1)*3*2**(p-1)].reset_index(drop=True))
        run_samples[-1] = run_samples[-1].T.iloc[df3.iloc[r]].reset_index(drop=True).T

        sample_Matrices = [run_samples[-1].iloc[:,m*int(len(run_samples[-1].T)/n):(m+1)*int(len(run_samples[-1].T)/n)] 
                           for m in range(n)]

        mixed_Matrices = []
        for j in range(k):
            mixed_Matrices.append(sample_Matrices[0].copy())
            mixed_Matrices[-1][j]=sample_Matrices[-1][k+j]

        elementary_effects_dic = {f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions}
        for f in functions:
            for j in range(k):
                elementary_effects_dic[f.__name__][j] = 0.5*(f(sample_Matrices[0].iloc[:2**p])-f(mixed_Matrices[j].iloc[:2**p]))**2

            C_T = elementary_effects_dic[f.__name__].expanding(1).mean()
            Var = f(sample_Matrices[0]).expanding(1).var(ddof=0)
            T = (C_T.T/Var).T
            AE_r = np.abs(T - AE_df.loc[f.__name__])
            MAE_r = AE_r.mean(axis=1)
            MAE_r.index=(MAE_r.index+1)*(k+1)
            MAE_dic[f.__name__][r]=MAE_r

            
    writer = pd.ExcelWriter(str(2**p)+' Saltenis_New_Estimator_comparison.xlsx', engine='xlsxwriter')

    for mk in MAE_dic.keys():
        MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)
    
    writer.save()